In [10]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import pickle
from datetime import datetime

In [2]:
#read sql db
engine = create_engine("sqlite:///sold_houses.db")

In [3]:
engine.table_names()

<ipython-input-3-d72bc75a6b62>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['houses']

In [4]:
df = pd.read_sql('SELECT * FROM houses;', engine)

In [5]:
df.columns

Index(['index', 'sale_type', 'sold_date', 'property_type', 'address', 'city',
       'state', 'zip_code', 'sold_price', 'beds', 'baths', 'location', 'sqft',
       'lot_size', 'year_built', 'days_on_market', 'price_sqft', 'HOA',
       'status', 'open_house_start', 'open_house_end', 'url', 'source', 'MLS',
       'favorite', 'interested', 'latitude', 'longitude'],
      dtype='object')

In [6]:
#drop values where no price is available
df =df[df['sold_price'].notna()]
df.shape

(110923, 28)

In [7]:
#Impute missing values for sold date
df['sold_date'] = df['sold_date']. replace('nan', np.nan)
df['sold_date'] = df['sold_date'].fillna(method='ffill')

In [8]:
def new_const(row):
    if row['year_built'] >= 2021:
        return 1
    else:
        return 0  
df['new_const'] = df.apply(new_const, axis=1)

In [9]:
df = df.drop('index', 1)

In [16]:
#convert to datetime
df['sold_date'] = df['sold_date'].astype(str)
def convert_date(row):
    return datetime.strptime(row["sold_date"], "%Y-%m-%d")
df['sold_date'] = df.apply(convert_date, axis=1)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110923 entries, 0 to 111635
Data columns (total 28 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   sale_type         110923 non-null  object        
 1   sold_date         110923 non-null  datetime64[ns]
 2   property_type     110923 non-null  object        
 3   address           110498 non-null  object        
 4   city              110875 non-null  object        
 5   state             110923 non-null  object        
 6   zip_code          110921 non-null  object        
 7   sold_price        110923 non-null  float64       
 8   beds              105120 non-null  float64       
 9   baths             103955 non-null  float64       
 10  location          105994 non-null  object        
 11  sqft              92330 non-null   float64       
 12  lot_size          83937 non-null   float64       
 13  year_built        94662 non-null   float64       
 14  days

In [18]:
df.to_csv("houses_data_v2.csv")

In [20]:
with open('houses_data.pkl', 'wb') as f:
    pickle.dump(df, f)